# Tarea 3 MongoDB, arXiv
##### Integrantes:
Bruno Morici, ROL: 202373555-8,
Martin Aranda, ROL: 202373021-1

## Primera Etapa
Instalación de dependencias

In [1]:
# Instalamos dependencias
!pip install pymongo
!pip install tqdm


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Segunda Etapa
Conexión a la BD "arxiv_db" y creación de colección "articles"

In [3]:
from pymongo import MongoClient

# Conectamos la BD usando localhost y los puertos mapeados
client = MongoClient(
    "mongodb://localhost:27017,localhost:27018,localhost:27019/?replicaSet=rs0&readPreference=primary")

db = client["arxiv_db"] # Nombre de la BD
collection = db["articles"] # Nombre de la coleccion

try:
    # Intenta obtener información del servidor
    server_info = client.server_info()
    print("Conexión exitosa a MongoDB!")
    print(f"Versión de MongoDB: {server_info['version']}")
    print(f"Dirección de conexión: {client.primary}") # Puedes usar client.primary para ver el primario después de la inicialización
except Exception as e:
    print(f"Error de conexión: {e}")

Conexión exitosa a MongoDB!
Versión de MongoDB: 8.0.9
Dirección de conexión: ('mongo2', 27017)


## Tercera Etapa
Procesamiento del dataset hacia la BD (ejecutar una sola vez, luego los datos quedan en la BD)

Observación: El dataset tiene un JSON en cada línea, no es una lista de JSON's. Recorremos cada JSON, lo parseamos y guardamos en una lista de data (demora unos 6 minutos)

In [ ]:
import json
from tqdm import tqdm # Para mostrar una barra de progreso

# Cargar y procesar el archivo JSON
print("Iniciando carga de datos a MongoDB...")

# Carga por lotes de 1000 en 1000
def load_data_batch(batch_size=1000):
    batch = []
    total_docs = 0
    
    # Primero contamos las lineas para la barra de progreso
    with open('arxiv-metadata-oai-snapshot.json', 'r', encoding='utf-8') as f:
        total_lines = sum(1 for _ in f)
    
    # Insertamos los datos en el batch
    with open('arxiv-metadata-oai-snapshot.json', 'r', encoding='utf-8') as f:
        for line in tqdm(f, total=total_lines, desc="Cargando datos"):
            batch.append(json.loads(line)) # Parseamos la linea a un JSON
            
            # Si el batch excede el maximo, lo vaciamos e insertamos los datos en la BD
            if len(batch) >= batch_size:
                collection.insert_many(batch)
                total_docs += len(batch)
                batch = []
        
        # Insertar el ultimo lote, en caso de que el ultimo lote sean menos de 1000
        if batch:
            collection.insert_many(batch)
            total_docs += len(batch)
    
    print(f"\nTotal de documentos insertados: {total_docs}")

load_data_batch() # Llamamos la funcion creada anteriormente

# 3. Verificacion final
print("\nResumen de la base de datos:")
print(f"- Colección: {collection.name}")
print(f"- Documentos totales: {collection.count_documents({})}")
print(f"- Ejemplo de documento: {collection.find_one()}")


Iniciando carga de datos a MongoDB...


Cargando datos:  83%|████████▎ | 2283999/2744489 [30:16<06:06, 1257.43it/s] 


KeyboardInterrupt: 

: 